# Task 1: Complete Data Merge and Exploratory Data Analysis

**Team Members:** Mitali, Blessing, Liliane, Mwai

**Objective:**
- Load customer_social_profiles and customer_transactions datasets
- Perform comprehensive EDA with required visualizations
- Clean and preprocess data
- Merge datasets with justified logic
- Engineer features for product prediction
- Save merged dataset for modeling

**Note:** This notebook builds upon initial merge work and completes all rubric requirements.

## 1. Import Required Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy import stats

# Machine learning preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 2. Load Datasets

In [ ]:
# Load datasets
social_df = pd.read_csv('../data/raw/customer_social_profiles.csv')
transactions_df = pd.read_csv('../data/raw/customer_transactions.csv')

print(f"Social Profiles Shape: {social_df.shape}")
print(f"Transactions Shape: {transactions_df.shape}")
print("\nDatasets loaded successfully!")

## 3. Initial Data Exploration

### 3.1 Social Profiles Dataset

In [ ]:
print("=" * 80)
print("SOCIAL PROFILES DATASET - INITIAL EXPLORATION")
print("=" * 80)

print("\nFirst 5 rows:")
display(social_df.head())

print("\nDataset Info:")
social_df.info()

print("\nSummary Statistics:")
display(social_df.describe())

print("\nColumn Names:")
print(social_df.columns.tolist())

In [ ]:
# Check for missing values
print("\nMissing Values in Social Profiles:")
missing_social = social_df.isnull().sum()
missing_social_pct = (missing_social / len(social_df)) * 100
missing_info_social = pd.DataFrame({
    'Missing Count': missing_social,
    'Percentage': missing_social_pct
})
display(missing_info_social[missing_info_social['Missing Count'] > 0])

if missing_social.sum() == 0:
    print("No missing values found!")

# Check for duplicates
duplicates_social = social_df.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates_social}")

### 3.2 Transactions Dataset

In [ ]:
print("=" * 80)
print("TRANSACTIONS DATASET - INITIAL EXPLORATION")
print("=" * 80)

print("\nFirst 5 rows:")
display(transactions_df.head())

print("\nDataset Info:")
transactions_df.info()

print("\nSummary Statistics:")
display(transactions_df.describe())

print("\nColumn Names:")
print(transactions_df.columns.tolist())

In [ ]:
# Check for missing values
print("\nMissing Values in Transactions:")
missing_trans = transactions_df.isnull().sum()
missing_trans_pct = (missing_trans / len(transactions_df)) * 100
missing_info_trans = pd.DataFrame({
    'Missing Count': missing_trans,
    'Percentage': missing_trans_pct
})
display(missing_info_trans[missing_info_trans['Missing Count'] > 0])

if missing_trans.sum() == 0:
    print("No missing values found!")

# Check for duplicates
duplicates_trans = transactions_df.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates_trans}")

## 4. Data Cleaning

### 4.1 Handle Missing Values

In [ ]:
# Create copies for cleaning
social_clean = social_df.copy()
transactions_clean = transactions_df.copy()

# Handle missing values if any exist
# Strategy: Fill numeric with median, categorical with mode, or drop rows with critical missing data

if social_clean.isnull().sum().sum() > 0:
    print("Handling missing values in social profiles...")
    # Add handling logic based on columns
    for col in social_clean.columns:
        if social_clean[col].isnull().sum() > 0:
            if social_clean[col].dtype in ['int64', 'float64']:
                social_clean[col].fillna(social_clean[col].median(), inplace=True)
                print(f"  - Filled {col} with median")
            else:
                social_clean[col].fillna(social_clean[col].mode()[0], inplace=True)
                print(f"  - Filled {col} with mode")
else:
    print("No missing values to handle in social profiles")

if transactions_clean.isnull().sum().sum() > 0:
    print("\nHandling missing values in transactions...")
    for col in transactions_clean.columns:
        if transactions_clean[col].isnull().sum() > 0:
            if transactions_clean[col].dtype in ['int64', 'float64']:
                transactions_clean[col].fillna(transactions_clean[col].median(), inplace=True)
                print(f"  - Filled {col} with median")
            else:
                transactions_clean[col].fillna(transactions_clean[col].mode()[0], inplace=True)
                print(f"  - Filled {col} with mode")
else:
    print("No missing values to handle in transactions")

print("\nMissing value handling complete!")

### 4.2 Remove Duplicates

In [ ]:
# Remove duplicates
rows_before_social = len(social_clean)
social_clean = social_clean.drop_duplicates()
rows_removed_social = rows_before_social - len(social_clean)

rows_before_trans = len(transactions_clean)
transactions_clean = transactions_clean.drop_duplicates()
rows_removed_trans = rows_before_trans - len(transactions_clean)

print(f"Social Profiles - Rows removed: {rows_removed_social}")
print(f"Transactions - Rows removed: {rows_removed_trans}")
print(f"\nFinal shapes:")
print(f"  Social: {social_clean.shape}")
print(f"  Transactions: {transactions_clean.shape}")

### 4.3 Standardize Column Names and Fix Data Types

In [ ]:
# Standardize customer ID columns for merging
print("Standardizing column names...")

# Rename columns to have consistent customer_id
if 'customer_id_new' in social_clean.columns:
    social_clean.rename(columns={'customer_id_new': 'customer_id'}, inplace=True)
    print("  - Renamed 'customer_id_new' to 'customer_id' in social profiles")

if 'customer_id_legacy' in transactions_clean.columns:
    transactions_clean.rename(columns={'customer_id_legacy': 'customer_id'}, inplace=True)
    print("  - Renamed 'customer_id_legacy' to 'customer_id' in transactions")

# Extract numeric ID from social profiles if needed (e.g., "CUST-150" -> 150)
if social_clean['customer_id'].dtype == 'object':
    print("\nExtracting numeric customer_id from social profiles...")
    social_clean['customer_id'] = social_clean['customer_id'].str.extract('(\d+)').astype(int)
    print("  - Conversion complete")

print("\nColumn standardization complete!")
print(f"Social customer_id dtype: {social_clean['customer_id'].dtype}")
print(f"Transactions customer_id dtype: {transactions_clean['customer_id'].dtype}")

## 5. Exploratory Data Analysis (EDA)

### Required: At least 3 labeled visualizations showing distributions, outliers, and correlations

### 5.1 Plot 1: Distribution of Numerical Features

In [ ]:
# Get numerical columns from both datasets
social_numeric = social_clean.select_dtypes(include=[np.number]).columns.tolist()
trans_numeric = transactions_clean.select_dtypes(include=[np.number]).columns.tolist()

# Remove customer_id from visualization
social_numeric = [col for col in social_numeric if col != 'customer_id']
trans_numeric = [col for col in trans_numeric if col != 'customer_id']

print("Numerical columns in social profiles:", social_numeric)
print("Numerical columns in transactions:", trans_numeric)

# Plot distributions for social profiles
if len(social_numeric) > 0:
    n_cols = min(3, len(social_numeric))
    n_rows = (len(social_numeric) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if len(social_numeric) > 1 else [axes]
    
    for idx, col in enumerate(social_numeric):
        social_clean[col].hist(bins=30, ax=axes[idx], edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel(col, fontsize=10)
        axes[idx].set_ylabel('Frequency', fontsize=10)
        axes[idx].grid(True, alpha=0.3)
    
    # Hide empty subplots
    for idx in range(len(social_numeric), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Social Profiles - Distribution of Numerical Features', 
                 fontsize=14, fontweight='bold', y=1.002)
    plt.show()

# Plot distributions for transactions
if len(trans_numeric) > 0:
    n_cols = min(3, len(trans_numeric))
    n_rows = (len(trans_numeric) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if len(trans_numeric) > 1 else [axes]
    
    for idx, col in enumerate(trans_numeric):
        transactions_clean[col].hist(bins=30, ax=axes[idx], edgecolor='black', alpha=0.7, color='green')
        axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel(col, fontsize=10)
        axes[idx].set_ylabel('Frequency', fontsize=10)
        axes[idx].grid(True, alpha=0.3)
    
    # Hide empty subplots
    for idx in range(len(trans_numeric), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Transactions - Distribution of Numerical Features', 
                 fontsize=14, fontweight='bold', y=1.002)
    plt.show()

print("\nInterpretation:")
print("- The histograms show the distribution patterns of numerical features")
print("- Look for skewness, multimodality, or unusual patterns")
print("- Normal distributions appear bell-shaped, while skewed distributions lean left or right")

### 5.2 Plot 2: Box Plots for Outlier Detection

In [ ]:
# Box plots for outlier detection - Social Profiles
if len(social_numeric) > 0:
    fig, axes = plt.subplots(1, len(social_numeric), figsize=(5*len(social_numeric), 6))
    if len(social_numeric) == 1:
        axes = [axes]
    
    for idx, col in enumerate(social_numeric):
        axes[idx].boxplot(social_clean[col].dropna(), vert=True)
        axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
        axes[idx].set_ylabel(col, fontsize=10)
        axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('Social Profiles - Outlier Detection (Box Plots)', 
                 fontsize=14, fontweight='bold', y=1.002)
    plt.show()

# Box plots for transactions
if len(trans_numeric) > 0:
    fig, axes = plt.subplots(1, len(trans_numeric), figsize=(5*len(trans_numeric), 6))
    if len(trans_numeric) == 1:
        axes = [axes]
    
    for idx, col in enumerate(trans_numeric):
        axes[idx].boxplot(transactions_clean[col].dropna(), vert=True)
        axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
        axes[idx].set_ylabel(col, fontsize=10)
        axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('Transactions - Outlier Detection (Box Plots)', 
                 fontsize=14, fontweight='bold', y=1.002)
    plt.show()

print("\nInterpretation:")
print("- Box plots show the median (middle line), quartiles (box boundaries), and outliers (dots)")
print("- Outliers are data points that fall significantly outside the typical range")
print("- Long whiskers or many outlier points may indicate extreme values that need investigation")

### 5.3 Plot 3: Correlation Heatmap

In [ ]:
# Correlation heatmap for social profiles
if len(social_numeric) > 1:
    plt.figure(figsize=(10, 8))
    correlation_social = social_clean[social_numeric].corr()
    sns.heatmap(correlation_social, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=1, fmt='.2f', cbar_kws={"shrink": 0.8})
    plt.title('Correlation Heatmap - Social Profiles', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print("\nStrong correlations in social profiles:")
    # Find strong correlations (abs value > 0.5, excluding diagonal)
    strong_corr = []
    for i in range(len(correlation_social.columns)):
        for j in range(i+1, len(correlation_social.columns)):
            if abs(correlation_social.iloc[i, j]) > 0.5:
                strong_corr.append((correlation_social.columns[i], 
                                   correlation_social.columns[j], 
                                   correlation_social.iloc[i, j]))
    if strong_corr:
        for col1, col2, corr_val in strong_corr:
            print(f"  - {col1} vs {col2}: {corr_val:.3f}")
    else:
        print("  - No strong correlations found (threshold: |r| > 0.5)")

# Correlation heatmap for transactions
if len(trans_numeric) > 1:
    plt.figure(figsize=(10, 8))
    correlation_trans = transactions_clean[trans_numeric].corr()
    sns.heatmap(correlation_trans, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=1, fmt='.2f', cbar_kws={"shrink": 0.8})
    plt.title('Correlation Heatmap - Transactions', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print("\nStrong correlations in transactions:")
    strong_corr = []
    for i in range(len(correlation_trans.columns)):
        for j in range(i+1, len(correlation_trans.columns)):
            if abs(correlation_trans.iloc[i, j]) > 0.5:
                strong_corr.append((correlation_trans.columns[i], 
                                   correlation_trans.columns[j], 
                                   correlation_trans.iloc[i, j]))
    if strong_corr:
        for col1, col2, corr_val in strong_corr:
            print(f"  - {col1} vs {col2}: {corr_val:.3f}")
    else:
        print("  - No strong correlations found (threshold: |r| > 0.5)")

print("\nInterpretation:")
print("- Values close to +1 indicate strong positive correlation")
print("- Values close to -1 indicate strong negative correlation")
print("- Values close to 0 indicate weak or no linear relationship")
print("- Strong correlations may indicate redundant features or important relationships")

### 5.4 Additional EDA: Categorical Variables

In [ ]:
# Analyze categorical variables
social_categorical = social_clean.select_dtypes(include=['object']).columns.tolist()
trans_categorical = transactions_clean.select_dtypes(include=['object']).columns.tolist()

print("Categorical columns in social profiles:", social_categorical)
print("Categorical columns in transactions:", trans_categorical)

# Value counts for categorical variables
if len(social_categorical) > 0:
    print("\n" + "="*80)
    print("SOCIAL PROFILES - CATEGORICAL VARIABLE ANALYSIS")
    print("="*80)
    for col in social_categorical:
        print(f"\n{col}:")
        print(social_clean[col].value_counts())
        print(f"Unique values: {social_clean[col].nunique()}")

if len(trans_categorical) > 0:
    print("\n" + "="*80)
    print("TRANSACTIONS - CATEGORICAL VARIABLE ANALYSIS")
    print("="*80)
    for col in trans_categorical:
        print(f"\n{col}:")
        print(transactions_clean[col].value_counts())
        print(f"Unique values: {transactions_clean[col].nunique()}")

## 6. Data Merge

### 6.1 Merge Strategy and Justification

In [ ]:
print("="*80)
print("MERGE STRATEGY")
print("="*80)

print("\nJoin Key: customer_id")
print("\nJustification:")
print("- Both datasets contain customer_id as the linking field")
print("- This allows us to combine social profile information with transaction history")
print("- Enables analysis of how social media behavior relates to purchasing patterns")

print("\nMerge Type: INNER JOIN")
print("\nReason:")
print("- We only want customers who have both social profiles AND transaction history")
print("- This ensures complete data for product recommendation modeling")
print("- Customers without either component cannot be properly analyzed")

print("\nPre-merge Analysis:")
print(f"- Unique customers in social profiles: {social_clean['customer_id'].nunique()}")
print(f"- Unique customers in transactions: {transactions_clean['customer_id'].nunique()}")
print(f"- Customers in both datasets: {len(set(social_clean['customer_id']) & set(transactions_clean['customer_id']))}")

### 6.2 Aggregate Transaction Data

In [ ]:
# Since one customer may have multiple transactions, aggregate them
print("Aggregating transaction data per customer...")

# Check how many transactions per customer
trans_per_customer = transactions_clean.groupby('customer_id').size()
print(f"\nAverage transactions per customer: {trans_per_customer.mean():.2f}")
print(f"Max transactions per customer: {trans_per_customer.max()}")
print(f"Min transactions per customer: {trans_per_customer.min()}")

# Aggregate transactions
agg_functions = {}
for col in trans_numeric:
    if col in transactions_clean.columns:
        agg_functions[col] = ['mean', 'sum', 'std']

# Add categorical aggregations
for col in trans_categorical:
    if col in transactions_clean.columns and col != 'customer_id':
        agg_functions[col] = lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]

transactions_agg = transactions_clean.groupby('customer_id').agg(agg_functions).reset_index()

# Flatten column names
transactions_agg.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                            for col in transactions_agg.columns.values]

print(f"\nAggregated transactions shape: {transactions_agg.shape}")
print("\nAggregated columns:")
print(transactions_agg.columns.tolist())

### 6.3 Perform Merge

In [ ]:
# Perform inner join
merged_df = pd.merge(
    social_clean,
    transactions_agg,
    on='customer_id',
    how='inner'
)

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nRows in social profiles: {len(social_clean)}")
print(f"Rows in transactions (aggregated): {len(transactions_agg)}")
print(f"Rows after merge: {len(merged_df)}")
print(f"\nData loss: {len(social_clean) - len(merged_df)} customers from social profiles")

print("\nFirst 5 rows of merged dataset:")
display(merged_df.head())

### 6.4 Validate Merge

In [ ]:
print("="*80)
print("MERGE VALIDATION")
print("="*80)

# Check for unexpected nulls
print("\nMissing values after merge:")
missing_merged = merged_df.isnull().sum()
if missing_merged.sum() > 0:
    display(missing_merged[missing_merged > 0])
else:
    print("No missing values!")

# Check for duplicates
duplicates_merged = merged_df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates_merged}")

# Verify unique customers
unique_customers = merged_df['customer_id'].nunique()
print(f"\nUnique customers in merged dataset: {unique_customers}")
print(f"Total rows: {len(merged_df)}")

if unique_customers == len(merged_df):
    print("Validation passed: One row per customer")
else:
    print("Warning: Multiple rows per customer detected")

print("\nMerged dataset info:")
merged_df.info()

## 7. Feature Engineering

Create new features to improve product recommendation model performance

In [ ]:
print("Creating engineered features...\n")

# Create a copy for feature engineering
merged_engineered = merged_df.copy()

# Feature 1: Customer value score (if purchase amount exists)
if any('purchase_amount' in col.lower() for col in merged_engineered.columns):
    purchase_cols = [col for col in merged_engineered.columns if 'purchase_amount' in col.lower()]
    if purchase_cols:
        sum_col = [col for col in purchase_cols if 'sum' in col.lower()]
        if sum_col:
            merged_engineered['customer_value_score'] = merged_engineered[sum_col[0]]
            print("Created: customer_value_score (total purchase amount)")

# Feature 2: Average transaction value
if any('purchase_amount_mean' in col.lower() for col in merged_engineered.columns):
    mean_cols = [col for col in merged_engineered.columns if 'purchase_amount_mean' in col.lower()]
    if mean_cols:
        merged_engineered['avg_transaction_value'] = merged_engineered[mean_cols[0]]
        print("Created: avg_transaction_value")

# Feature 3: Transaction consistency (std deviation normalized)
if any('purchase_amount_std' in col.lower() for col in merged_engineered.columns) and \
   any('purchase_amount_mean' in col.lower() for col in merged_engineered.columns):
    std_cols = [col for col in merged_engineered.columns if 'purchase_amount_std' in col.lower()]
    mean_cols = [col for col in merged_engineered.columns if 'purchase_amount_mean' in col.lower()]
    if std_cols and mean_cols:
        merged_engineered['transaction_consistency'] = (
            merged_engineered[std_cols[0]] / (merged_engineered[mean_cols[0]] + 1)
        )
        print("Created: transaction_consistency (coefficient of variation)")

# Feature 4: Social engagement score (if relevant columns exist)
social_engagement_cols = [col for col in merged_engineered.columns 
                          if any(keyword in col.lower() for keyword in 
                                ['follower', 'like', 'post', 'engagement'])]
if social_engagement_cols:
    # Normalize and sum engagement metrics
    scaler = StandardScaler()
    engagement_scaled = scaler.fit_transform(merged_engineered[social_engagement_cols].fillna(0))
    merged_engineered['social_engagement_score'] = engagement_scaled.mean(axis=1)
    print(f"Created: social_engagement_score (from {len(social_engagement_cols)} metrics)")

# Feature 5: Customer rating score (if exists)
rating_cols = [col for col in merged_engineered.columns if 'rating' in col.lower()]
if rating_cols:
    merged_engineered['avg_customer_rating'] = merged_engineered[rating_cols[0]]
    print("Created: avg_customer_rating")

print(f"\nTotal features after engineering: {merged_engineered.shape[1]}")
print("\nNew feature columns:")
new_features = [col for col in merged_engineered.columns if col not in merged_df.columns]
print(new_features)

## 8. Define Target Variable

Identify or create the target variable for product recommendation

In [ ]:
# Check for product-related columns
product_cols = [col for col in merged_engineered.columns if 'product' in col.lower()]
print("Product-related columns found:")
print(product_cols)

# If product_category exists, use it as target
if product_cols:
    target_col = product_cols[0]
    print(f"\nTarget variable: {target_col}")
    print(f"\nTarget distribution:")
    print(merged_engineered[target_col].value_counts())
    
    # Visualize target distribution
    plt.figure(figsize=(10, 6))
    merged_engineered[target_col].value_counts().plot(kind='bar', edgecolor='black')
    plt.title(f'Distribution of Target Variable: {target_col}', 
              fontsize=14, fontweight='bold')
    plt.xlabel(target_col, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("\nNo obvious product column found. Manual target creation may be needed.")

## 9. Final Dataset Summary

In [ ]:
print("="*80)
print("FINAL DATASET SUMMARY")
print("="*80)

print(f"\nShape: {merged_engineered.shape}")
print(f"Number of customers: {merged_engineered['customer_id'].nunique()}")
print(f"Number of features: {merged_engineered.shape[1]}")

print("\nColumn types:")
print(merged_engineered.dtypes.value_counts())

print("\nMemory usage:")
print(f"{merged_engineered.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nSample of final dataset:")
display(merged_engineered.head(10))

## 10. Save Processed Dataset

In [ ]:
# Save to processed folder
output_path = '../data/processed/merged_customer_data.csv'
merged_engineered.to_csv(output_path, index=False)

print(f"Dataset saved successfully to: {output_path}")
print(f"\nFile size: {merged_engineered.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Rows: {len(merged_engineered)}")
print(f"Columns: {len(merged_engineered.columns)}")

# Verify save
test_load = pd.read_csv(output_path)
print(f"\nVerification: Successfully loaded {len(test_load)} rows from saved file")

## 11. Key Findings and Insights

In [ ]:
print("="*80)
print("KEY FINDINGS AND INSIGHTS")
print("="*80)

print("\n1. DATA QUALITY:")
print(f"   - Started with {len(social_df)} social profiles and {len(transactions_df)} transactions")
print(f"   - After cleaning and merging: {len(merged_engineered)} customers with complete data")
print(f"   - Data retention rate: {len(merged_engineered)/len(social_df)*100:.1f}%")

print("\n2. MERGE STRATEGY:")
print("   - Used inner join on customer_id to ensure complete records")
print("   - Aggregated multiple transactions per customer into summary statistics")
print("   - Result: One row per customer with comprehensive features")

print("\n3. FEATURE ENGINEERING:")
print(f"   - Created {len(new_features)} new features from raw data")
print("   - Combined social media metrics with transaction patterns")
print("   - Features ready for machine learning model training")

print("\n4. NEXT STEPS:")
print("   - Use this dataset for product recommendation model (Task 4)")
print("   - Apply classification/regression algorithms")
print("   - Evaluate model performance with accuracy, F1-score, and loss metrics")

print("\n" + "="*80)
print("TASK 1 COMPLETE - Dataset ready for modeling!")
print("="*80)